In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras as tf_keras

In [8]:
# 데이터 준비

columns = ["mpg", "cylinders", "displacement", "horsepower", "weight", "acceleration", "model_year", "origin"]

mpg_df = pd.read_csv('drive/MyDrive/Colab Notebooks/data-files/auto-mpg.data', names=columns,
                     sep=" ", na_values="?", comment="\t", skipinitialspace=True)
mpg_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [ ]:
# 데이터 준비 2
# 1. mpg 컬럼을 y로 나머지 컬럼을 x로 분할
# 2. origin 컬럼을 one-hot-encoding 처리
# 3. 회귀모델 만들기 : mpg(y)를 예측하는 회귀모델
# 4. 훈련데이터 평가

In [11]:
mpg_df['origin'].value_counts()

,count
origin,
1,249
3,79
2,70
